# Guided exercise 5: Creating new types

As we show in last guided exercise in Haskell we can create our own types, actually this is quite common. There are three different ways to do it: `type`, `data` and `newtype`.

## Creating new types with data: single constructor with parameters

We have already seen how to use `type`, and `data` with constructors with no parameters. The most generic case is to use `data` with parameters: `data Name = Constructor1 Type11 Type12 ... | Contstructor2 Type21 Type22  | ... deriving (TypeClass1, TypeClass2...)`

It is a good idea to derive at least from `Show` to be able to show the results.

When we have only one constructor it is common that the name of the constructor and the name of the type are the same, but it is not compulsory.

In [11]:
-- Two new types with data
data NewType = NewType Int Float deriving Show
data NewType2 = Constructor Int Int deriving Show
-- Data of them
NewType 2 4
Constructor 3 4
-- Which are the types of the constructors?
:t NewType
:t Constructor
-- They are functions that return their types!!
-- That is the reason why different types cannot have constructors with the same name

NewType 2 4.0

Constructor 3 4

NewType :: Int -> Float -> NewType

Constructor :: Int -> Int -> NewType2

**Exercise 1.** Create definition of the `Date` type of the weekly exercises using `data`:
1. A type to represent a Date with day (Int), month (Int), year (Int)
2. A function `isleap date` to return if the year of a Date is a leap year
3. A function `rightMonth date` to check if a month number is correct
4. A function `rightDay date` to check if a day number is correct for that month of that Date
5. A function `fixDate date` that if the month or the day is incorrect, fixes them, the month will be 1 and the day will be 1 too.
6. A function `monthName date` that returns the name of the month of that date
7. A function `showDate date` that returns "day of monthName of the leap/not leap year"
8. A function `nextDay date` that returns the next Date of a given one
9. A function `isPrevious date1 date2` that returns if date1 is previous to date2

In [12]:
-- 1
data Date = Date Int Int Int deriving Show

In [13]:
-- 2
isleap::Date -> Bool
isleap (Date _ _ y) = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)

-- 3
rightMonth::Date -> Bool
rightMonth (Date _ m _) = m > 0 && m < 13

-- 4 
rightDay::Date -> Bool
rightDay (Date d m y)
    | d < 1 || d > 31 = False
    | m `elem` [1, 3, 5, 7, 8, 10, 12] = True
    | m == 2 = d < 29 || (d == 29 && isleap (Date d m y))
    | m `elem` [4, 6, 9, 11] = d <= 30
    | otherwise = error "Wrong month"

-- 5
fixDate::Date -> Date
fixDate (Date d m y)
    | rightMonth (Date d m y) = if rightDay (Date d m y) then Date d m y else Date 1 m y
    | otherwise = if rightDay (Date d 1 y) then Date d 1 y else Date 1 1 y
 
-- 6
monthName::Date -> String
monthName (Date d m y) = if rightMonth (Date d m y) 
                        then months !! (m - 1) 
                        else error "Wrong month"
                        where months = ["january", "february", "march", "april", "may", "june", "july", "august", 
                                        "september", "october", "november", "december"]
                        
-- 7
showDate::Date -> String
showDate (Date d m y) = show d ++ " of " ++ monthName (Date d m y) ++ " of the " ++ leap ++ show y
    where leap = if isleap (Date d m y) then "leap " else "non leap "
    
-- 8 
nextDay::Date -> Date
nextDay (Date d m y) 
    | rightDay (Date (d + 1) m y) = Date (d + 1) m y
    | rightMonth (Date 1 (m + 1) y) = Date 1 (m + 1) y
    | otherwise = Date 1 1 (y + 1)
 
-- 9
isPrevious::Date -> Date -> Bool
isPrevious (Date d1 m1 y1) (Date d2 m2 y2) 
    | y1 > y2 = False
    | y1 < y2 = True
    | m1 > m2 = False
    | m1 < m2 = True
    | d1 < d2 = True
    | otherwise = False

In [14]:
-- Testing code
--2
isleap (Date 1 1 1900)
isleap (Date 1 1 2000)
isleap (Date 1 1 2012)
-- 3
rightMonth (Date 1 1 1900)
rightMonth (Date 1 12 1900)
rightMonth (Date 1 13 1900)
-- 4
rightDay (Date 1 12 1900)
rightDay (Date 31 12 1900)
rightDay (Date 32 12 1900)
rightDay (Date 30 11 1900)
rightDay (Date 29 2 1900)
rightDay (Date 29 2 2000)
-- 5
fixDate (Date 32 12 1900)
fixDate (Date 30 11 1900)
fixDate (Date 29 2 1900)
fixDate (Date 5 24 2021)
fixDate (Date 41 24 2012)
-- 6
monthName (Date 1 3 2020)
-- 7
showDate (Date 1 3 2020)
-- 8
nextDay (Date 1 1 2022)
nextDay (Date 28 2 2022)
nextDay (Date 31 12 2022)
-- 9
isPrevious (Date 1 1 2022) (Date 28 2 2022)
isPrevious (Date 27 2 2022) (Date 28 2 2022)
isPrevious (Date 27 1 2022) (Date 27 1 2022)

False

True

True

True

True

...

Date 1 3 2022

Date 1 1 2023

True

True

False

## Using @ for pattern matching

When declaring a function with an algebraic data type (when using `data`) with parameters we can use `@` to create an alias of the input of the function to avoid repeating it into the body:

In [15]:
-- without @
showDate::Date -> String
showDate (Date d m y) = show d ++ " of " ++ monthName (Date d m y) ++ " of the " ++ leap ++ show y
    where leap = if isleap (Date d m y) then "leap " else "non leap "

    
-- with @
showDate'::Date -> String
showDate' theDate@(Date d m y) = show d ++ " of " ++ monthName theDate ++ " of the " ++ leap ++ show y
    where leap = if isleap theDate then "leap " else "non leap "

**Exercise 2.** Change the `fixDate` function to use this notation.

In [16]:
fixDate::Date -> Date
fixDate dt@(Date d m y)
    | rightMonth dt = if rightDay dt then dt else Date 1 m y
    | otherwise = if rightDay (Date d 1 y) then Date d 1 y else Date 1 1 y

In [17]:
fixDate (Date 32 12 1900)
fixDate (Date 30 11 1900)
fixDate (Date 29 2 1900)
fixDate (Date 5 24 2021)
fixDate (Date 41 24 2012)

Date 1 12 1900

Date 30 11 1900

Date 1 2 1900

Date 5 1 2021

Date 1 1 2012

## Record syntax

When a new type is created with `data` a name can be given to each parameter to make it clearer what their meaning is. This is known as record syntax. Doing it that way also modifies how the values are shown.

In [18]:
data Date = Date {day::Int, month::Int, year::Int} deriving Show

In [19]:
-- In the functions we can use the old style
isleap::Date -> Bool
isleap (Date _ _ y) = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)

-- Or the new one
isleap'::Date -> Bool
isleap' Date {day, month, year} = year `mod` 4 == 0 && (year `mod` 100 /= 0 || year `mod` 400 == 0)

In [20]:
-- Notice that now show works in a different way

-- We can create values in the old way
Date 1 1 1900
-- But also using the names, in this case any order is valid
Date {year = 2000, day = 12, month = 9}
-- Both ways also to invoke a function
isleap' (Date 1 1 2022)
isleap' Date {year = 2000, day = 12, month = 9}

Date {day = 1, month = 1, year = 1900}

Date {day = 12, month = 9, year = 2000}

False

True

**Exercise 3.** Use record syntax to create a Person type with name, surname, id and age

In [21]:
data Person = Person {name::String, surname::String, id::String, age::Int} deriving Show
-- Testing it
Person "Pepe" "Perez" "444802840F" 33
Person {name = "Pepe", surname = "Perez", id = "444802840F", age = 33}

Person {name = "Pepe", surname = "Perez", id = "444802840F", age = 33}

Person {name = "Pepe", surname = "Perez", id = "444802840F", age = 33}

An alternative is to use `type` to create synonyms:

In [22]:
type Name = String
type Surname = String
type Id = String
type Age = Int
data Person = Person Name Surname Id Age deriving Show

--We can use the new types in the functions to make them easier to undestand
personName::Person -> Name
personName (Person n _ _ _) = n

personName (Person "Juan" "Perez" "192031" 12)
Person "Juan" "Perez" "192031" 12

"Juan"

Person "Juan" "Perez" "192031" 12

## data: multiple constructors with parameters

We can have different consructors with different parameters.

**Exercise 4.** Create a second constructor where the `month` is `String`. Check the types of the constructors. Modify the `isleap` and `rightMonth` functions so they work with both constructors. 

In [23]:
-- We use the record syntax in one of them just to show it can be done
data Date = Date1 Int Int Int | Date2 {day::Int, month::String, year::Int} deriving Show

-- 2
isleap::Date -> Bool
isleap (Date1 _ _ y) = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)
isleap (Date2 _ _ y) = y `mod` 4 == 0 && (y `mod` 100 /= 0 || y `mod` 400 == 0)

-- 3
rightMonth::Date -> Bool
rightMonth (Date1 _ m _) = m > 0 && m < 13
rightMonth (Date2 _ m _) = m `elem` ["january", "february", "march", "april", "may", "june", "july", "august", 
                                        "september", "october", "november", "december"]

In [24]:
:t Date1
:t Date2

Date1 :: Int -> Int -> Int -> Date

Date2 :: Int -> String -> Int -> Date

In [25]:
-- Testing code
--2
isleap (Date1 1 1 1900)
isleap (Date1 1 1 2000)
isleap (Date1 1 1 2012)
isleap (Date2 1 "january" 1900)
isleap (Date2 1 "january" 2000)
isleap (Date2 1 "january" 2012)
-- 3
rightMonth (Date1 1 1 1900)
rightMonth (Date1 1 12 1900)
rightMonth (Date1 1 13 1900)
rightMonth (Date2 1 "january" 1900)
rightMonth (Date2 1 "december" 1900)
rightMonth (Date2 1 "januaryyy" 1900)

False

True

True

False

True

...

True

False

True

True

False

## Deriving from typeclasses

We can use `deriving` to mark that a type is an instance of a given typeclass. This makes it automatically implement all the typeclass functions.

In [26]:
-- We make person to derive also from Eq and Ord
data Person = Person {name::String, surname::String, id::String, age::Int} deriving (Show, Eq, Ord)

In [27]:
-- Now we can use the equality and ordering operators
Person "Juan" "Perez" "192031" 12 == Person "Juan" "Perez" "192031" 12
Person "Juan" "Perez" "192031" 12 == Person "Juan" "Perez" "192031" 33
-- It compares first and first, then second and second until an answer is found
Person "Juan" "Perez" "192031" 12 > Person "Juan" "Perez" "129021" 29
Person "Juan" "Perez" "192031" 12 > Person "Santiago" "Perez" "129021" 9

True

False

True

False

When we derive from a typeclass we use the by-default implementations of the functions, but they can be overrided, either using the original one or giving a totally new implementation. In this case we say the type is an *instance* of the typeclass and we don't need to derive from it using `deriving`. To do it, we use:
```
instance typeclass newType where 
    new_definition_of_function
```

It is not compulsory to implement all the methods, if we give no definition, the by-default one will be used.

In [45]:
data Person = Person {name::String, surname::String, id::String, age::Int} deriving Show
-- We are redefining == based on the original == 
instance Eq Person where
    (Person _ _ id1 _) == (Person _ _ id2 _) = id1 == id2

In [41]:
Person "Antonio" "Gonzalez" "192031" 12 == Person "Juan" "Perez" "192031" 21

True

**Exercise 5.** Redefine the >, >=, <= and < operators for Person so they compare the ages

In [48]:
instance Ord Person where
    (Person _ _ _ ag1) < (Person _ _ _ ag2) = ag1 < ag2
    (Person _ _ _ ag1) > (Person _ _ _ ag2) = ag1 > ag2
    (Person _ _ _ ag1) <= (Person _ _ _ ag2) = ag1 <= ag2
    (Person _ _ _ ag1) >= (Person _ _ _ ag2) = ag1 >= ag2

In [49]:
Person "Juan" "Perez" "192031" 12 >= Person "Antonio" "Gonzalez" "00031" 21
Person "Juan" "Perez" "192031" 21 >= Person "Antonio" "Gonzalez" "00031" 21

False

True

**Exercise 6.** Redefine the show function of Person type so it shows something like: Juan Perez with id 192031 is 12 years old

In [38]:
instance Show Person where
    show (Person n s i a) = n ++ " " ++  s ++ " with id " ++ i ++ " is " ++ show a ++ " years old"

In [39]:
Person "Juan" "Perez" "192031" 12

Juan Perez with id 192031 is 12 years old

## General parameters

The parameters of a constructor can be general ones

In [65]:
data Operands a = Binary a a | Unary a
:t Binary
:t Unary

addition:: Num a => Operands a -> a
addition (Unary a) = a
addition (Binary b c) = b + c

Binary :: forall a. a -> a -> Operands a

Unary :: forall a. a -> Operands a

In [67]:
addition (Unary 4)
addition (Binary 2 4)

4

6